# Simple Agent Example

I this example, we show how to create a small world with a single (Drape) agent which can walk around. Note that the update only uses addition and multiplication.

In [1]:
import os, sys, curses, torch, six, itertools, collections
import numpy as np

from campx import things
from campx.ascii_art import ascii_art_to_game, Partial
from campx import engine


import syft as sy
from syft.core.frameworks.torch import utils

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
# alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
# james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)
me.add_worker(bob)
# me.add_workers([bob, alice, james])
# bob.add_workers([me, alice, james])
# alice.add_workers([me, bob, james])
# james.add_workers([me, bob, alice])

In [2]:
GAME_ART = ['#####',
             '#A* #',
             '#*#*#',
             '# * #',
             '#####']

In [3]:
class AgentDrape(things.Drape):
    """A Drape that just moves an agent around the board using a probablility vector"""

    def update(self, actions, board, layers, backdrop, all_things, the_plot):
        del board, layers, backdrop, all_things  # unused

        if actions is None: return  # No work needed to make the first observation.
        
        act = actions
        
        b = self.curtain
        
        left = torch.cat([b[:,1:],b[:,:1]], dim=1)
        right = torch.cat([b[:,-1:],b[:,:-1]], dim=1)
        up= torch.cat([b[1:],b[:1]], dim=0)
        down = torch.cat([b[-1:],b[:-1]], dim=0)
        stay = b

        b = (act[0] * left) + (act[1] * right) + (act[2] * up) + (act[3] * down) + (act[4] * stay)
        self.curtain.set_(b)
        
        the_plot.add_reward(1)  # Give ourselves a point for moving.

In [4]:
def make_game():
  """Builds and returns a Hello World game."""
  return ascii_art_to_game(
      GAME_ART,
      what_lies_beneath=' ',
      drapes={'A': AgentDrape},
      z_order='A')
game = make_game()

In [5]:
board, reward, discount = game.its_showtime()

In [6]:
# this will move the 65 around the board ([left, right, up, down, stay])

board, reward, discout = game.play([1,0,0,0,0])

board.board


  0   0   0   0   0
 65   0   0   0   0
  0   0   0   0   0
  0   0   0   0   0
  0   0   0   0   0
[syft.core.frameworks.torch.tensor.LongTensor of size 5x5]

In [7]:
game._the_plot

{}

In [8]:
# this is what you'd use as input for the agent
board.layered_board


(0 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

(1 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

(2 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

(3 ,.,.) = 
  0  0  0  0  0
  1  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
[syft.core.frameworks.torch.tensor.ByteTensor of size 4x5x5]